# Day 24: Arithmetic Logic Unit

Magic smoke starts leaking from the submarine's arithmetic logic unit (ALU). Without the ability to perform basic arithmetic and logic functions, the submarine can't produce cool patterns with its Christmas lights!

It also can't navigate. Or run the oxygen system.

Don't worry, though - you probably have enough oxygen left to give you enough time to build a new ALU.

## ALU

The ALU is a four-dimensional processing unit: it has integer variables w, x, y, and z. These variables all start with the value 0. The ALU also supports six instructions:

- inp a - Read an input value and write it to variable a.
- add a b - Add the value of a to the value of b, then store the result in variable a.
- mul a b - Multiply the value of a by the value of b, then store the result in variable a.
- div a b - Divide the value of a by the value of b, truncate the result to an integer, then store the result in variable a. (Here, "truncate" means to round the value toward zero.)
- mod a b - Divide the value of a by the value of b, then store the remainder in variable a. (This is also called the modulo operation.)
- eql a b - If the value of a and b are equal, then store the value 1 in variable a. Otherwise, store the value 0 in variable a.

In all of these instructions, a and b are placeholders; a will always be the variable where the result of the operation is stored (one of w, x, y, or z), while b can be either a variable or a number. Numbers can be positive or negative, but will always be integers.

The ALU has no jump instructions; in an ALU program, every instruction is run exactly once in order from top to bottom. The program halts after the last instruction has finished executing.

(Program authors should be especially cautious; attempting to execute div with b=0 or attempting to execute mod with a<0 or b<=0 will cause the program to crash and might even damage the ALU. These operations are never intended in any serious ALU program.)

In [ ]:
static char ToReg(int r)
    => (char)('w'+r);

enum Ins { inp, add, mul, div, mod, eql, debug };

record Op (Ins ins, int dest, int src, bool srcReg) {
    static public Op Parse(string l) {
        var parts = l.Split(' ');
        var ins = Enum.Parse<Ins>(parts[0]);
        if (ins == Ins.debug) return new Op(ins, 0, 0, false);
        var dest = (int)(parts[1][0]-'w');
        var src = 0;
        if (ins == Ins.inp || int.TryParse(parts[2], out src))
            return new Op(ins, dest, src, false);
        return new Op(ins, dest, parts[2][0]-'w', true);
    }

    string Source => srcReg ? $"{ToReg(src)}" : $"{src}";

    public override string ToString()
        => ins switch {
            Ins.debug => "debug",
            Ins.inp => $"inp {ToReg(dest)}",
            _ => $"{ins} {ToReg(dest)} {Source}"
        };
}


## Processing

In [ ]:
record State(int[] inp) {
    decimal[] vars = new decimal[4];

    public State Copy => new State(inp.ToArray()) { vars = vars.ToArray() };
    public State Pop(out decimal src) {
        src = inp[0];
        return new State(inp[1..].ToArray()) { vars = vars.ToArray() };
    }

    public void Set(int reg, decimal val) => vars[reg] = val;
    public decimal Get(int reg) => vars[reg];
    
    public override string ToString()
        => string.Join(" ", 
            vars.Select((v,r) => $"{ToReg(r)}={v}")
                .Append(":")
                .Concat(inp.Select(i => $"{i}"))
        );
}

static decimal Calc(Ins? ins, decimal left, decimal right)
    => ins switch {
            Ins.add => left + right,
            Ins.mul => left * right,
            Ins.div => Math.Truncate(left / right),
            Ins.mod => left % right,
            Ins.eql => left == right ? 1 : 0,
            _ => right
        };

static decimal Min(Ins? ins, decimal left, decimal right)
    => ins switch {
        Ins.inp or Ins.eql or Ins.mod => 0,
        Ins.add => left + right,
        Ins.mul => left * right,
        Ins.div => right == 0 ? 0 : left / right,
        _ => -1
    };

static decimal Max(Ins? ins, decimal left, decimal right)
    => ins switch {
        Ins.inp => 9,
        Ins.add => left + right,
        Ins.mul => left * right,
        Ins.div => right == 0 ? 0 : left / right,
        Ins.mod => right - 1,
        Ins.eql => 1,
        _ => -1
    };

static State Process(State s, Op o) {
    if (o.ins == Ins.debug) {
        Console.WriteLine(s);
        return new State(s.inp.ToArray());
    }

    var res = s.Copy;
    var left = s.Get(o.dest);
    decimal right = 0;
    if (o.ins == Ins.inp) {
        res = s.Pop(out right);
    } else {
        right = o.srcReg ? s.Get(o.src) : o.src;
    }

    var val = Calc(o.ins, left, right);

    res.Set(o.dest, val);
    return res;
}

## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-24.sample");
var input = File.ReadAllLines(@"day-24.input");

var ops = input.Select(Op.Parse).ToArray();

// ops.Select(o => $"{o}");
ops.Length

252

## Part 1

Once you have built a replacement ALU, you can install it in the submarine, which will immediately resume what it was doing when the ALU failed: validating the submarine's model number. To do this, the ALU will run the MOdel Number Automatic Detector program (MONAD, your puzzle input).

Submarine model numbers are always fourteen-digit numbers consisting only of digits 1 through 9. The digit 0 cannot appear in a model number.

When MONAD checks a hypothetical fourteen-digit model number, it uses fourteen separate inp instructions, each expecting a single digit of the model number in order of most to least significant. (So, to check the model number 13579246899999, you would give 1 to the first inp instruction, 3 to the second inp instruction, 5 to the third inp instruction, and so on.) This means that when operating MONAD, each input instruction should only ever be given an integer value of at least 1 and at most 9.

Then, after MONAD has finished running all of its instructions, it will indicate that the model number was valid by leaving a 0 in variable z. However, if the model number was invalid, it will leave some other non-zero value in z.

MONAD imposes additional, mysterious restrictions on model numbers, and legend says the last copy of the MONAD documentation was eaten by a tanuki. You'll need to figure out what MONAD does some other way.

To enable as many submarine features as possible, find the largest valid fourteen-digit model number that contains no 0 digits. What is the largest model number accepted by MONAD?

### Expr and Registers

In [ ]:
static string Paren(bool paren, string expr)
    => paren ? $"({expr})" : expr;

record Expr(int op, Ins? ins, Expr dest, decimal? val, Expr src, decimal max, decimal min) {
    static public Expr Val(int op, decimal val)
        => new Expr(op, null, null, val, null, val, val);
    static public Expr Op(int op, Ins ins, Expr dest, Expr src)
        => new Expr(op, ins, dest, null, src, Min(ins, dest.min, src.min), Max(ins, dest.max, src.max));
    public Expr ToVal(decimal val)
        => this with { ins = null, val = val, min = val, max = val };
    public Expr ToOp(Expr left, Expr right)
        => this with { dest = left, src = right
                , min = Min(ins, left.min, right.min) 
                , max = Max(ins, left.max, right.max)
            };
    public bool Is(decimal value)
        => ins == null && val == value;
    string Dest(int d) 
        => dest == null ? ""
            : $" [{dest.Describe(d)}]";
    string Src(int d)
        => src == null ? $" {val}" 
            : $" [{src.Describe(d)}]";
    string MinMax
        => $" {min}..{max}";
    string Describe(int d)
        => d < 0 ? "..." 
            : ins == null ? $"{op}: {val}"
            : $"{op}: {ins}{Dest(d-1)}{Src(d-1)}{MinMax}";
    public override string ToString()
        => Describe(4);
    public string Show (int pri)
        => ins switch {
            null => $"{val}",
            Ins.inp => $"i{val}",
            Ins.add => Paren(pri != 0, $"{dest.Show(0)}+{src?.Show(0)}{val}"),
            Ins.mul => Paren(pri != 2, $"{dest.Show(2)}*{src?.Show(2)}{val}"),
            Ins.div => Paren(pri != 2, $"{dest.Show(2)}/{src?.Show(2)}{val}"),
            Ins.mod => Paren(pri != 2, $"{dest.Show(2)}%{src?.Show(2)}{val}"),
            Ins.eql => Paren(pri != 1, $"{dest.Show(1)}={src?.Show(1)}{val}"),
            _ => $"?{ins} ",
        };
    public decimal Process(int[] inputs)
        => ins switch {
            null => val.Value,
            Ins.inp => inputs[(int)val.Value],
            Ins.add or Ins.mul or Ins.div or Ins.mod or
            Ins.eql => Calc(ins, dest.Process(inputs), src.Process(inputs)),
            _ => -1,
        };
}

record Registers {
    int inp;
    Expr[] vars  = new []{ Expr.Val(-1, 0), Expr.Val(-1, 0), Expr.Val(-1, 0), Expr.Val(-1, 0), };

    public Registers Copy => new Registers { inp = inp, vars = vars.ToArray() };
    public Registers Pop(out int src) {
        src = inp;
        return new Registers { inp = inp + 1, vars = vars.ToArray() };
    }

    public void Set(int reg, Expr val) => vars[reg] = val;
    public Expr Get(int reg) => vars[reg];

    public string Show
        => string.Join("\n", vars
            .Select((v,r) => $"{ToReg(r)}={v.Show(0)}")
            .Prepend($"Inputs {inp}")
        );

    public override string ToString()
        => string.Join("\n", vars
                .Select((v,r) => $"{ToReg(r)}={v}")
                .Prepend($"Inputs {inp}")
            );
}

### Simplification

In [ ]:

static Expr Distribute(Expr val, Ins? op, Expr times, Expr left, Expr right) {
    var dest = Simplify(val, left, times);
    var src = Simplify(val, right, times);
    var result = val with { ins = op };
    result = result.ToOp(dest, src);
    // Console.WriteLine($"R: {times}*({left} {op} {right}) -> {result}\n");
    return result;
}

static Expr Product(Expr val, decimal times, Expr left, Expr right) {
    if (left.ins == null) return val.ToOp(right, left.ToVal(left.val.Value * times));
    return val.ToOp(left, right.ToVal(right.val.Value * times));
}

static Expr SimplifyMul(Expr val, Expr left, Expr right) {
    if (left.ins == null) {
        if (left.val == 0) return val.ToVal(0);
        if (left.val == 1) return right;        
        // if (right.ins is Ins.add or Ins.div)
        //     return Distribute(val, right.ins, left, right.dest,  right.src);
        // if (right.ins == Ins.mul && (right.dest.ins == null || right.src.ins == null))
        //     return Product(val, left.val.Value, right.dest, right.src);
    }
    if (right.ins == null) {
        if (right.val == 0) return val.ToVal(0);
        if (right.val == 1) return left;
        // if (left.ins is Ins.add or Ins.div)
        //     return Distribute(val, left.ins, right, left.dest, left.src);
        // if (left.ins == Ins.mul && (left.dest.ins == null || left.src.ins == null))
        //     return Product(val, right.val.Value, left.dest, left.src);
    }
    return val.ToOp(left, right);
}

static Expr SimplifyMod(Expr val, Expr left, Expr right) {
    if (right.ins == null) {
        if (left.ins == Ins.add && left.dest.ins == Ins.mul && left.dest.src.Is(right.val.Value))
            return left.src;
        if (right.val > left.min)
            return left;
    }
    return val.ToOp(left, right);
}

static Expr Simplify(Expr val, Expr left, Expr right) {
    if (left.ins == null && right.ins == null) {
        var num = Calc(val.ins, left.val.Value, right.val.Value);
        return val.ToVal(num);
    }

    return val.ins switch {
        Ins.add when left.Is(0) => right,
        Ins.add when right.Is(0) => left,

//        Ins.mul => SimplifyMul(val, left, right),
        Ins.mul when left.Is(0) => val.ToVal(0),
        Ins.mul when right.Is(0) => val.ToVal(0),
        Ins.mul when left.Is(1) => right,
        Ins.mul when right.Is(1) => left,

        Ins.div when right.Is(1) => left,
        Ins.mod => SimplifyMod(val, left, right),
        Ins.eql when left.max < right.min || left.min > right.max
            => val.ToVal(0),

        _ => val.ToOp(left, right),
    };
}

### Analysis

In [ ]:
static Registers Analyze(Registers a, (Op o,int i) t) {
    if (t.o.ins == Ins.debug) {
        Console.WriteLine($"{t.i}: {a}");
        return a;
    }
    var res = a.Copy;
    var val = new Expr(t.i, t.o.ins, null, null, null, 0, 0);
    if (val.ins == Ins.inp) {
        res = a.Pop(out var inp);
        val = val with { val = inp, min = 1, max = 9 };
    } else  {
        var left = a.Get(t.o.dest);
        var right = t.o.srcReg ? a.Get(t.o.src) : Expr.Val(t.i, t.o.src);
        var simpler = Simplify(val, left, right);
        val = simpler ?? val.ToOp(left, right);
    }

    res.Set(t.o.dest, val);
    return res;
}

var idx = 90;
var expr = ops[..idx].Select((o,i) => (o,i)).Aggregate(new Registers(), Analyze);
Console.WriteLine(expr);
Console.WriteLine(ops[idx]);
expr.Show

Inputs 5
w=72: inp 4 1..9
x=79: eql [78: eql [77: add [54: inp 3 1..9] [77: -8] -7..1] [72: inp 4 1..9] 0..1] [79: 0] 0..1
y=88: mul [87: add [72: inp 4 1..9] [87: 1] 2..10] [79: eql [78: eql [77: add [54: inp 3 1..9] [77: -8] -7..1] [72: inp 4 1..9] 0..1] [79: 0] 0..1] 0..10
z=89: add [84: mul [76: div [71: add [66: mul [...] [...] 64480..210704] [54: inp 3 1..9] 64481..210713] [76: 26] 2480.0384615384615384615384615..8104.346153846153846153846154] [83: add [82: mul [81: 25] [79: eql [...] [...] 0..1] 0..25] [83: 1] 1..26] 2480.0384615384615384615384615..210713.00000000000000000000000] [88: mul [87: add [72: inp 4 1..9] [87: 1] 2..10] [79: eql [78: eql [77: add [...] [...] -7..1] [72: inp 4 1..9] 0..1] [79: 0] 0..1] 0..10] 2480.0384615384615384615384615..210723.00000000000000000000000
inp w


Inputs 5
w=i4
x=((i3+-8)=i4=0)
y=((i4+1)*((i3+-8)=i4=0))
z=((((((((i0+2)*26)+i1+16)*26)+i2+9)*26)+i3)/26*((25*((i3+-8)=i4=0))+1))+((i4+1)*((i3+-8)=i4=0))

In [ ]:
var t = (o: ops[idx], i: idx);
var val = new Expr(t.i, t.o.ins, null, null, null, 0, 0);
var left = expr.Get(t.o.dest);
var right = t.o.srcReg ? expr.Get(t.o.src) : Expr.Val(t.i, t.o.src);
Console.WriteLine($"{t}\n L: {left}\n R: {right}");

Simplify(val, left, right).ToString()

(inp w, 90)
 L: 72: inp 4 1..9
 R: 90: 0


90: inp [72: inp 4 1..9] [90: 0] 0..9

## Part 1 Processed

In [ ]:
record Display {
    Dictionary<Expr,int> counts = new();
    Dictionary<Expr,decimal> values = new();
    HashSet<Expr> shown = new();

    public void Collect(Expr expr) {
        if (expr?.ins == null || expr.ins == Ins.inp) return;
        if (counts.TryGetValue(expr, out var count))
            counts[expr] = count + 1;
        else {
            counts.Add(expr, 1);
            Collect(expr.dest);
            Collect(expr.src);
        }
    }

    public decimal Value(Expr expr, int[] inputs) {
        if (values.TryGetValue(expr, out var result))
            return result;

        result = expr.ins switch {
            null => expr.val.Value,
            Ins.inp => inputs[(int)expr.val.Value],
            Ins.add or Ins.mul or Ins.div or Ins.mod or
            Ins.eql => Calc(expr.ins, Value(expr.dest, inputs), Value(expr.src, inputs)),
        };

        if (counts.TryGetValue(expr, out var count) && count > 1)
            values[expr] = result;
        return result;
    }

    bool Function(Expr expr, string label) {
        if (counts.TryGetValue(expr, out var count) && count > 1) {
            if (!shown.Contains(expr)) {
                if (values.TryGetValue(expr, out var value))
                    label += $" {value} %{value%26}";
                Console.WriteLine(label + " => " + Expression(expr, 0));
                shown.Add(expr);
            }
            return false;
        }
        return true;
    }

    string Operation(string op, int pri, Expr dest, Expr src) {
        var left = $"f_{dest.op}";
        var right = $"f_{src.op}";
    
        if (Function(dest, left)) left = Expression(dest, pri);
        if (Function(src, right)) right = Expression(src, pri);
    
        return left + op + right;
    }

    string Expression(Expr expr, int pri)
        => expr.ins switch {
            null => $"{expr.val}",
            Ins.inp => $"i_{expr.val}",
            Ins.add => Paren(pri != 0, Operation("+", 0, expr.dest, expr.src)),
            Ins.mul => Paren(pri != 2, Operation("*", 2, expr.dest, expr.src)),
            Ins.div => Paren(pri != 2, Operation("/", 2, expr.dest, expr.src)),
            Ins.mod => Paren(pri != 2, Operation("%", 2, expr.dest, expr.src)),
            Ins.eql => Paren(pri != 1, Operation("=", 1, expr.dest, expr.src)),
            _ => $"?{expr.ins} ",
        };

    public void Show(Expr expr) 
        => Console.WriteLine(Expression(expr, 0));
}


In [ ]:
//95491959997964 too low
//98491959997994 

var start = new State(new[] {
        9, 8, 4, 9, 1, 9, 5, 
        9, 9, 9, 7, 9, 9, 4
    });
var processed = ops.Aggregate(start, Process);
Console.WriteLine(processed);

var analyzed = ops.Select((o,i) => (o,i)).Aggregate(new Registers(), Analyze);
var z = analyzed.Get(3);
display(z.Process(start.inp));

var exprs = new Display();
exprs.Collect(z);
display(exprs.Value(z, start.inp));
exprs.Show(z)
// exprs.OrderByDescending(kv => kv.Value).Select(kv => $"{kv.Value} - {kv.Key.Show(0)}")

w=4 x=0 y=0 z=0 :


0

0

f_79 0 %0 => ((i_3+-8)=i_4=0)
f_105 21 %21 => i_5+12
f_115 0 %0 => ((f_105+-16)=i_6=0)
f_125 8073 %13 => ((((((((((((i_0+2)*26)+i_1+16)*26)+i_2+9)*26)+i_3)/26*((25*f_79)+1))+((i_4+1)*f_79))*26)+f_105)/26*((25*f_115)+1))+((i_6+6)*f_115)
f_133 0 %0 => (((f_125%26)+-4)=i_7=0)
f_159 12 %12 => i_8+3
f_169 0 %0 => ((f_159+-3)=i_9=0)
f_195 16 %16 => i_10+9
f_205 0 %0 => ((f_195+-7)=i_11=0)
f_215 310 %24 => (((((((((f_125/26*((25*f_133)+1))+((i_7+6)*f_133))*26)+f_159)/26*((25*f_169)+1))+((i_9+5)*f_169))*26)+f_195)/26*((25*f_205)+1))+((i_11+3)*f_205)
f_223 0 %0 => (((f_215%26)+-15)=i_12=0)
f_233 11 %11 => (f_215/26*((25*f_223)+1))+((i_12+2)*f_223)
f_241 0 %0 => ((f_233+-7)=i_13=0)
(f_233/26*((25*f_241)+1))+((i_13+3)*f_241)


```
i_3  -8 = i_4  => ( 9, 1 )
i_5  -4 = i_6  => ( 9, 5 )
i_2  +5 = i_7  => ( 4, 9 )
i_8     = i_9  => ( 9, 9 )
i_10 +2 = i_11 => ( 7, 9 )
i_1  +1 = i_12 => ( 8, 9 )
i_0  -5 = i_13 => ( 9, 4 )
```

# Part 2

As the submarine starts booting up things like the Retro Encabulator, you realize that maybe you don't need all these submarine features after all.

What is the smallest model number accepted by MONAD?

```
i_0  -5 = i_13 => ( 6, 1 )
i_1  +1 = i_12 => ( 1, 2 )
i_2  +5 = i_7  => ( 1, 6 )
i_3  -8 = i_4  => ( 9, 1 )
i_5  -4 = i_6  => ( 5, 1 )
i_8     = i_9  => ( 1, 1 )
i_10 +2 = i_11 => ( 1, 3 )
```

In [ ]:
// 61191516111321

var start = new State(new[] {
    6, 1, 1, 9, 1, 5, 1, 
    6, 1, 1, 1, 3, 2, 1
});
var processed = ops.Aggregate(start, Process);
Console.WriteLine(processed);

var analyzed = ops.Select((o,i) => (o,i)).Aggregate(new Registers(), Analyze);
var z = analyzed.Get(3);
display(z.Process(start.inp));

var exprs = new Display();
exprs.Collect(z);
display(exprs.Value(z, start.inp));
exprs.Show(z)

w=1 x=0 y=0 z=0 :


0

0

f_79 0 %0 => ((i_3+-8)=i_4=0)
f_105 17 %17 => i_5+12
f_115 0 %0 => ((f_105+-16)=i_6=0)
f_125 5860 %10 => ((((((((((((i_0+2)*26)+i_1+16)*26)+i_2+9)*26)+i_3)/26*((25*f_79)+1))+((i_4+1)*f_79))*26)+f_105)/26*((25*f_115)+1))+((i_6+6)*f_115)
f_133 0 %0 => (((f_125%26)+-4)=i_7=0)
f_159 4 %4 => i_8+3
f_169 0 %0 => ((f_159+-3)=i_9=0)
f_195 10 %10 => i_10+9
f_205 0 %0 => ((f_195+-7)=i_11=0)
f_215 225 %17 => (((((((((f_125/26*((25*f_133)+1))+((i_7+6)*f_133))*26)+f_159)/26*((25*f_169)+1))+((i_9+5)*f_169))*26)+f_195)/26*((25*f_205)+1))+((i_11+3)*f_205)
f_223 0 %0 => (((f_215%26)+-15)=i_12=0)
f_233 8 %8 => (f_215/26*((25*f_223)+1))+((i_12+2)*f_223)
f_241 0 %0 => ((f_233+-7)=i_13=0)
(f_233/26*((25*f_241)+1))+((i_13+3)*f_241)
